In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
from pathlib import Path
from lightkurve import KeplerLightCurveFile
import math
from gatspy.periodic import *